In [1]:
import numpy as np
from numpy import load

with load('/home/admin/rnd_test/FloBaRoID/model/youbot_arm_only.urdf.trajectory.npz')as data:
    q= data['q']

In [2]:
data = np.load('/home/admin/rnd_test/FloBaRoID/model/youbot_arm_only.urdf.trajectory.npz')
print(data['a'])
print("\n")
print(data['b'])
print(data['q'])
print(data['wf'])
print(data['nf'])
print(data['use_deg'])
print(data['static'])

[[-0.01843954 -1.35100414 -1.38460642 -1.49999526]
 [ 1.47889483 -0.68627869 -1.49898959  0.62888016]
 [-0.87304354  1.49305501  0.01446198  1.32713111]
 [-0.80310965 -0.27268255  0.20038992 -0.08897228]
 [ 0.32868034  1.47174043  0.3566105  -0.43804508]]


[[ 0.55884867  1.49311535  0.32678246  0.36818675]
 [ 1.32197464  0.23963626  1.49311535 -1.49976685]
 [-0.06066548  0.60247197 -0.8886058  -0.0830268 ]
 [-0.92282281  1.49311535  1.46462059  1.37916413]
 [-0.47232994  1.49311535 -1.49944466  1.14680166]]
[ 1.61959383  1.69281924 -0.47206474  2.86613349  1.98916314]
0.749359503834
[4 4 4 4 4]
0
False


In [3]:
a=np.array([[-0.35707, 0.21389, 0.43313, 0.05424], [-0.37936, 0.19053, 0.54725, 0.53816], [0.37673, 0.00059, -0.39666, 0.50074], [0.8947, 1.5, -0.19924, 0.22437], [-0.37039, 0.33709, -0.30782, -0.96412]])
b=np.array([[0.80867, 1.06223, -0.18519, 0.09188], [-0.11051, 0.211, 0.36583, 0.36593], [0.25166, 1.25223, 1.5, 0.98509], [0.18492, -1.4371, 0.98988, -0.47763], [-0.90683, 0.5881, 0.84402, 1.47759]])
q=np.array([0.5, 0.5, -0.5, 0.5, 0.5])
wf=1.27555600367
nf=np.array([4, 4, 4 ,4 ,4])
static=data['static']
use_deg=data['use_deg']

np.savez('try_traj_1.npz', a=a, b=b,nf=nf,q=q,static=static,use_deg=use_deg,wf=wf)
wf 1.61206752708
a [[0.26219, 0.36268, 0.32304, 0.64628], [-0.31021, -0.02746, -0.02615, -0.78478], [0.79597, 0.76411, 0.73664, -0.29413], [0.25069, 0.20837, 0.42928, 0.59341], [0.03639, -0.02684, 0.1727, 0.54239]]
b [[0.55636, -0.25348, 0.51058, 0.67728], [-0.00702, 0.3182, 0.36555, 0.09737], [0.2612, 0.26642, 1.45066, 0.75685], [0.7791, -0.36174, 0.4394, -1.20312], [-0.68709, 0.4479, -0.03858, 0.27756]]




SyntaxError: invalid syntax (<ipython-input-3-2eb100e79422>, line 10)

In [4]:
class OscillationGenerator(object):
    def __init__(self, w_f, a, b, q0, nf, use_deg):
        '''
        generate periodic oscillation from fourier series (Swevers, 1997)

        - w_f is the global pulsation (frequency is w_f / 2pi)
        - a and b are (arrays of) amplitudes of the sine/cosine
          functions for each joint
        - q0 is the joint angle offset (center of pulsation)
        - nf is the desired amount of coefficients for this fourier series
        '''
        self.w_f = float(w_f)
        self.a = a
        self.b = b
        self.use_deg = use_deg
        self.q0 = float(q0)
        if use_deg:
            self.q0 = np.deg2rad(self.q0)
        self.nf = nf

    def getAngle(self, t):
        #- t is the current time
        q = 0.0
        for l in range(1, self.nf+1):
            q += (self.a[l-1]/(self.w_f*l))*np.sin(self.w_f*l*t) - \
                 (self.b[l-1]/(self.w_f*l))*np.cos(self.w_f*l*t)
          
        q += self.nf*self.q0
        
        if self.use_deg:
            q = np.rad2deg(q)
        
        return q

    def getVelocity(self, t):
        dq = 0.0
        for l in range(1, self.nf+1):
            dq += self.a[l-1]*np.cos(self.w_f*l*t) + \
                  self.b[l-1]*np.sin(self.w_f*l*t)
        if self.use_deg:
            dq = np.rad2deg(dq)
        return dq

    def getAcceleration(self, t):
        ddq = 0.0
        for l in range(1, self.nf+1):
            ddq += -self.a[l-1]*self.w_f*l*np.sin(self.w_f*l*t) + \
                    self.b[l-1]*self.w_f*l*np.cos(self.w_f*l*t)
        if self.use_deg:
            ddq = np.rad2deg(ddq)
        return ddq
    
class Trajectory(object):
    ''' base trajectory class '''
    def getAngle(self, dof):
        raise NotImplementedError()

    def getVelocity(self, dof):
        raise NotImplementedError()

    def getAcceleration(self, dof):
        raise NotImplementedError()

    def getPeriodLength(self):
        raise NotImplementedError()

    def setTime(self, time):
        raise NotImplementedError()

    def wait_for_zero_vel(self, t_elapsed):
        raise NotImplementedError()

In [5]:
class PulsedTrajectory(Trajectory):
    ''' pulsating trajectory generator for one joint using fourier series from
        Swevers, Gansemann (1997). Gives values for one time instant (at the current
        internal time value)
    '''
    def __init__(self, dofs, use_deg=False):
        # type: (List, bool) -> None
        self.dofs = dofs
        self.oscillators = list()  # type: List[OscillationGenerator]
        self.use_deg = use_deg
        self.w_f_global = 1.0

   
    def initWithParams(self, a, b, q, nf, wf=None):
        ''' init with given params
            a - list of dof coefficients a
            b - list of dof coefficients b
            q - list of dof coefficients q_0
            nf - list of dof coefficients n_f
            (also see docstring of OscillationGenerator)
        '''

        if len(nf) != self.dofs or len(q) != self.dofs:
            raise Exception("Need DOFs many values for nf and q!")

        #for i in nf:
        #    if not ( len(a) == i and len(b) == i):
        #        raise Exception("Need nf many values in each parameter array value!")

        self.a = a
        self.b = b
        self.q = []
        self.nf = nf
        if wf:
            self.w_f_global = wf

        self.oscillators = list()
        for i in range(0, self.dofs):
            self.oscillators.append(OscillationGenerator(w_f = self.w_f_global, a = np.array(a[i]),
                                                         b = np.array(b[i]), q0 = q[i], nf = nf[i], use_deg = self.use_deg
                                                        ))
        return self

    def getAngle(self, dof):
        """ get angle at current time for joint dof """
        return self.oscillators[dof].getAngle(self.time)

    def getVelocity(self, dof):
        """ get velocity at current time for joint dof """
        return self.oscillators[dof].getVelocity(self.time)

    def getAcceleration(self, dof):
        """ get acceleration at current time for joint dof """
        return self.oscillators[dof].getAcceleration(self.time)

    def getPeriodLength(self):
        ''' get the period length of the oscillation in seconds '''
        return 2 * np.pi / self.w_f_global

    def setTime(self, time):
        '''set current time in seconds'''
        self.time = time

    def wait_for_zero_vel(self, t_elapsed):
        self.setTime(t_elapsed)
        if self.use_deg: thresh = 5.0
        else: thresh = np.deg2rad(5.0)
        return abs(self.getVelocity(0)) < thresh

In [6]:
from identification.data import Data
from identification.model import Model

def simulateTrajectory(trajectory, config):
    # type: (Dict, Trajectory, Model, np._ArrayLike) -> Tuple[Dict, Data]
    # generate data arrays for simulation and regressor building
    

   
    dofs=5
    
    trajectory_data = {}   # type: Dict[str, Union[List, np._ArrayLike]]
    trajectory_data['target_positions'] = []
    trajectory_data['target_velocities'] = []
    trajectory_data['target_accelerations'] = []
    trajectory_data['torques'] = []
    trajectory_data['times'] = []

    freq = 200
    for t in range(0, int(trajectory.getPeriodLength()*freq)):
        trajectory.setTime(t/freq)
        q = np.array([trajectory.getAngle(d) for d in range(dofs)])
        trajectory_data['target_positions'].append(q)
        #f.write("%s\n" % q)
        
        qdot = np.array([trajectory.getVelocity(d) for d in range(dofs)])
        trajectory_data['target_velocities'].append(qdot)

        qddot = np.array([trajectory.getAcceleration(d) for d in range(dofs)])
        trajectory_data['target_accelerations'].append(qddot)

        trajectory_data['times'].append(t/freq)
        trajectory_data['torques'].append(np.zeros(dofs))
   
    num_samples = len(trajectory_data['times'])
    
    #convert lists to numpy arrays
    trajectory_data['target_positions'] = np.array(trajectory_data['target_positions'])
    trajectory_data['positions'] = trajectory_data['target_positions']
    trajectory_data['target_velocities'] = np.array(trajectory_data['target_velocities'])
    trajectory_data['velocities'] = trajectory_data['target_velocities']
    trajectory_data['target_accelerations'] = np.array(trajectory_data['target_accelerations'])
    trajectory_data['accelerations'] = trajectory_data['target_accelerations']
    trajectory_data['torques'] = np.array(trajectory_data['torques'])
    trajectory_data['times'] = np.array(trajectory_data['times'])
    trajectory_data['measured_frequency'] = freq
    trajectory_data['base_velocity'] = np.zeros( (num_samples, 6) )
    trajectory_data['base_acceleration'] = np.zeros( (num_samples, 6) )

    trajectory_data['base_rpy'] = np.zeros( (num_samples, 3) )
    
    data = Data(config)
    data.init_from_data(trajectory_data)
    model.computeRegressors(data)
    
    return trajectory_data

In [7]:
from identification.data import Data
from identification.model import Model

def simulateTrajectory(trajectory, config):
    # type: (Dict, Trajectory, Model, np._ArrayLike) -> Tuple[Dict, Data]
    # generate data arrays for simulation and regressor building
    

   
    dofs=5
    
    trajectory_data = {}   # type: Dict[str, Union[List, np._ArrayLike]]
    trajectory_data['target_positions'] = []
    trajectory_data['target_velocities'] = []
    trajectory_data['target_accelerations'] = []
    trajectory_data['torques'] = []
    trajectory_data['times'] = []

    freq = 200
    for t in range(0, int(trajectory.getPeriodLength()*freq)):
        trajectory.setTime(t/freq)
        q = np.array([trajectory.getAngle(d) for d in range(dofs)])
        trajectory_data['target_positions'].append(q)
        #f.write("%s\n" % q)
        
        qdot = np.array([trajectory.getVelocity(d) for d in range(dofs)])
        trajectory_data['target_velocities'].append(qdot)

        qddot = np.array([trajectory.getAcceleration(d) for d in range(dofs)])
        trajectory_data['target_accelerations'].append(qddot)

        trajectory_data['times'].append(t/freq)
        trajectory_data['torques'].append(np.zeros(dofs))
   
    num_samples = len(trajectory_data['times'])
    
    #convert lists to numpy arrays
    trajectory_data['target_positions'] = np.array(trajectory_data['target_positions'])
    trajectory_data['positions'] = trajectory_data['target_positions']
    trajectory_data['target_velocities'] = np.array(trajectory_data['target_velocities'])
    trajectory_data['velocities'] = trajectory_data['target_velocities']
    trajectory_data['target_accelerations'] = np.array(trajectory_data['target_accelerations'])
    trajectory_data['accelerations'] = trajectory_data['target_accelerations']
    trajectory_data['torques'] = np.array(trajectory_data['torques'])
    trajectory_data['times'] = np.array(trajectory_data['times'])
    trajectory_data['measured_frequency'] = freq
    trajectory_data['base_velocity'] = np.zeros( (num_samples, 6) )
    trajectory_data['base_acceleration'] = np.zeros( (num_samples, 6) )

    trajectory_data['base_rpy'] = np.zeros( (num_samples, 3) )
    
    data = Data(config)
    data.init_from_data(trajectory_data)
    model.computeRegressors(data)
    
    return trajectory_data
    #- [0.0,5.89921]
    #- [0.0,2.70526] 
    #- [-5.18363,0.0]
    #- [0.0,3.57792]
    #- [0.0,5.75959]'''

In [8]:
dofs = 5
nf = 4

wf_min = 0.01
wf_max = 2.0
wf_init = 0.1

qmin = [-0.5]*dofs
qmax = [0.5]*dofs
qinit = [(0.5*(-0.436332)) + (0.5*(0.436332))]*dofs

amin = bmin = -1.5
amax = bmax = 1.5
ainit = [[0] * nf] * dofs
binit = [[0] * nf] * dofs
num_constraints = dofs*3 

def vecToParams(x):
        # convert vector of all solution variables to separate parameter variables
        nf = [4]*dofs
        wf = x[0]
        q = x[1:dofs+1]
        ab_len = dofs*nf[0]
        a = np.array(np.split(np.array(x[dofs+1:dofs+1+ab_len]), dofs))
        b = np.array(np.split(np.array(x[dofs+1+ab_len:dofs+1+ab_len*2]),dofs))
        return wf,q,a, b


In [9]:
from identification.model import Model
import iDynTree; iDynTree.init_helpers(); iDynTree.init_numpy_helpers()

import yaml


with open("/home/admin/rnd_test/FloBaRoID/configs/youbot_test.yaml", 'r') as stream:
    try:
        config = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

config['urdf'] = "/home/admin/rnd_test/FloBaRoID/model/youbot_arm.urdf"
model = Model(config, config['urdf'])



loaded model /home/admin/rnd_test/FloBaRoID/model/youbot_arm.urdf
# DOFs: 5
Joints: ['arm_joint_1', 'arm_joint_2', 'arm_joint_3', 'arm_joint_4', 'arm_joint_5']
# regressor outputs: 5
# links: 6 (+ 0 fake)
{0: 'arm_link_0', 1: 'arm_link_1', 2: 'arm_link_2', 3: 'arm_link_3', 4: 'arm_link_4', 5: 'arm_link_5'}
# params: 60 (60 will be identified)
loaded random structural regressor from /home/admin/rnd_test/FloBaRoID/model/youbot_arm.urdf.regressor.npz


In [10]:
from excitation.optimizer import plotter, Optimizer
import pyOpt
import random

def testBounds(x):
        #test variable bounds
        wf,a, b = vecToParams(x)
        wf_t = wf >= wf_min and wf <= wf_max
        q_t = np.all(q <= qmax) and np.all(q >= qmin)
        a_t = np.all(a <= amax) and np.all(a >= amin)
        b_t = np.all(b <= bmax) and np.all(b >= bmin)
        res = wf_t and a_t and b_t
        
        if not res:
            print("bounds violated")

        return res

def testConstraints(g):
        g = np.array(g)
        c_s = num_constraints 
        
        res = np.all(g[:c_s] <= 0.01)
        return res
    
def addVarsAndConstraints(opt_prob):
        # type: (pyOpt.Optimization) -> None
        ''' add variables, define bounds '''

        # w_f - pulsation
        opt_prob.addVar('wf', 'c', value=wf_init, lower=wf_min, upper=wf_max)

        # q - offsets
        for i in range(dofs):
            opt_prob.addVar('q_%d'%i,'c', value=qinit[i], lower=qmin[i], upper=qmax[i])
        # a, b - sin/cos params
        for i in range(dofs):
            for j in range(nf):
                opt_prob.addVar('a{}_{}'.format(i,j), 'c', value=ainit[i][j], lower=amin, upper=amax)
        for i in range(dofs):
            for j in range(nf):
                opt_prob.addVar('b{}_{}'.format(i,j), 'c', value=binit[i][j], lower=bmin, upper=bmax)

        # add constraint vars (constraint functions are in obfunc)
        opt_prob.addConGroup('g', num_constraints, type='i', lower=0.0, upper=np.inf)

def objectiveFunc( x, test=False):
        iter_cnt = 0
        iter_cnt += 1
        
        print("call #{}/{}".format(iter_cnt, iter_max))

        wf, q, a, b = vecToParams(x)
        print('wf {}'.format(wf))
        print('a {}'.format(np.round(a,5).tolist()))
        print('b {}'.format(np.round(b,5).tolist()))
        print('q {}'.format(np.round(q,5).tolist()))

        #input vars out of bounds, skip call
        if not testBounds(x):
            
            f = 1000.0
            g = [10.0]*num_constraints
            
            fail = 1.0
            return f, g, fail

        trajectory.initWithParams(a, b, q, nf, wf)
        f = np.linalg.cond(model.YBase)
        f1 = 0
        # add constraints  (later tested for all: g(n) <= 0)
        g = [1e10]*num_constraints
        jn = model.jointNames
        for n in range(dofs):
            f1 = 0
        # add constraints  (later tested for all: g(n) <= 0)
        g = [1e10]*num_constraints
        jn = model.jointNames
        
        
        for n in range(dofs):
            # check for joint limits
            # joint pos lower
            print("minma of trajectory",np.min(trajectory_data['positions'][:, n]))
            print("maxima of trajectory",np.max(trajectory_data['positions'][:, n]))
            
            if len(config['ovrPosLimit'])>n and config['ovrPosLimit'][n]:
                g[n] = np.deg2rad(config['ovrPosLimit'][n][0]) - np.min(trajectory_data['positions'][:, n])
                
            else:
                g[n] = limits[jn[n]]['lower'] - np.min(trajectory_data['positions'][:, n])
            # joint pos upper
            if len(config['ovrPosLimit'])>n and config['ovrPosLimit'][n]:
                g[dofs+n] = np.max(trajectory_data['positions'][:, n]) - np.deg2rad(config['ovrPosLimit'][n][1])
            else:
                g[dofs+n] = np.max(trajectory_data['positions'][:, n]) - limits[jn[n]]['upper']
            # max joint vel
            g[2*dofs+n] = np.max(np.abs(trajectory_data['velocities'][:, n])) - limits[jn[n]]['velocity']
         
        
        
def optimizeTrajectory():
        # type: () -> PulsedTrajectory
        # use non-linear optimization to find parameters for minimal
        # condition number trajectory

        # Instanciate Optimization Problem
        opt_prob = pyOpt.Optimization('Trajectory optimization', objectiveFunc)
        opt_prob.addObj('f')
        opt_prob = opt_prob

        addVarsAndConstraints(opt_prob)
        sol_vec = runOptimizer(opt_prob)
        print(sol_vec)

        sol_wf,sol_a, sol_b = vecToParams(sol_vec)
        
        trajectory.initWithParams(sol_a, sol_b,q, self.nf, sol_wf)

        return trajectory

def runOptimizer(opt_prob):
    sr = random.SystemRandom()
    mpi_size = 1
    mpi_rank = 0
    opt = pyOpt.ALPSO()  #augmented lagrange particle swarm optimization
    opt.setOption('stopCriteria', 0)   # stop at max iters
    opt.setOption('dynInnerIter', 1)   # dynamic inner iter number
    opt.setOption('maxInnerIter', 5)
    opt.setOption('maxOuterIter', config['globalOptIterations'])
    opt.setOption('printInnerIters', 1)
    opt.setOption('printOuterIters', 1)
    opt.setOption('SwarmSize', config['globalOptSize'])
    opt.setOption('xinit', 1)
    opt.setOption('seed', sr.random()*mpi_size) #(self.mpi_rank+1)/self.mpi_size)
    iter_max = opt.getOption('SwarmSize') * opt.getOption('maxInnerIter') * \
    opt.getOption('maxOuterIter') + opt.getOption('SwarmSize')
    iter_max = iter_max // mpi_size           

In [73]:
optimizeTrajectory()

None


TypeError: 'NoneType' object has no attribute '__getitem__'

In [1]:
import PyKDL

ImportError: No module named PyKDL